# 使用决策树对优惠券情况做出预测

##### 数据集是2016年1月1日至2016年6月30日之间真实线上线下消费行为，预测用户在2016年7月领取优惠券后15天以内的使用情况。 注意： 为了保护用户和商家的隐私，所有数据均作匿名处理，同时采用了有偏采样和必要过滤。

## 导包

In [8]:
import pandas as pd
import numpy as np

## 读取数据集

In [3]:
# 读取并查看info
train_data = pd.read_csv('ccf_offline/ccf_offline_stage1_train.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   User_id        int64  
 1   Merchant_id    int64  
 2   Coupon_id      float64
 3   Discount_rate  object 
 4   Distance       float64
 5   Date_received  float64
 6   Date           float64
dtypes: float64(4), int64(2), object(1)
memory usage: 93.7+ MB


In [4]:
# 查看前5条数据
train_data.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [5]:
# 数据预处理--丢弃缺失值
print(train_data.shape)
train_data = train_data.dropna(how='any')
print(train_data.shape)

(1754884, 7)
(67165, 7)


In [7]:
# Discount_rate是object类型的，object在pandas中代表字符串，
# 字符串类型不能输入模型中，所以需要改为数值类型
print(f"Discount_rate 类型：\n{train_data['Discount_rate'].unique()}")

Discount_rate 类型：
['20:1' '20:5' '30:5' '50:10' '10:5' '50:20' '100:10' '30:10' '50:5'
 '30:1' '100:30' '0.8' '200:30' '100:20' '10:1' '200:20' '0.95' '5:1'
 '100:5' '100:50' '50:1' '20:10' '150:10' '0.9' '200:50' '150:20' '150:50'
 '200:5' '300:30' '100:1' '200:10' '150:30' '0.85' '0.6' '0.5' '300:20'
 '200:100' '300:50' '150:5' '300:10' '0.75' '0.7' '30:20' '50:30']


In [10]:
# 将 Discount_rate 转化为数值特征
### 打折类型
### x:y 表示满 x 减 y          将 x:y 类型的字符串设为1
### [0,1] 表示折扣率           将 [0,1] 类型的字符串设为 0 

def getDiscountType(row):
    if ':' in row:
        return 1
    else:
        return 0
    
train_data['Discount_rate'] = train_data['Discount_rate'].apply(getDiscountType)

In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67165 entries, 6 to 1754880
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   User_id        67165 non-null  int64  
 1   Merchant_id    67165 non-null  int64  
 2   Coupon_id      67165 non-null  float64
 3   Discount_rate  67165 non-null  int64  
 4   Distance       67165 non-null  float64
 5   Date_received  67165 non-null  float64
 6   Date           67165 non-null  float64
dtypes: float64(4), int64(3)
memory usage: 4.1 MB


In [12]:
# 数据集添加一个label列 
### 标注标签Label  标注哪些样本是正样本 y=1，哪些是负样本 y = -1
### 预测目标：用户在领取优惠券之后 15 之内的消费情况
### (Date - Date_received <= 15) 表示领取优惠券且在15天内使用，即正样本，y = 1
### (Date - Date_received > 15)   表示领取优惠券未在15天内使用，即负样本，y = 0
### pandas 关于时间的教程 ```https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html```

def label(row):
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

train_data['label'] = train_data.apply(label, axis=1)

In [14]:
print(train_data['label'].value_counts())

1    57060
0    10105
Name: label, dtype: int64


## 导入模型和划分数据集的包

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 划分数据集，80%训练集，20%测试集

In [16]:
X_train, X_test, y_train, y_test = train_test_split(train_data.iloc[:, 1:],
                                                   train_data.iloc[:, 0],
                                                   test_size=0.2,
                                                   random_state=3)

In [21]:
# 查验训练样本的数量和类别分布
y_train.value_counts()

2751537    96
6641735    86
6929894    80
501441     59
2839484    56
           ..
5679946     1
2208346     1
1273834     1
3395905     1
4461556     1
Name: User_id, Length: 34984, dtype: int64

In [22]:
# 查验测试样本的数量和类别分布
y_test.value_counts()

6641735    27
2751537    22
2839484    15
2507268    14
501441     14
           ..
4284393     1
1242446     1
5408744     1
2577930     1
89464       1
Name: User_id, Length: 11405, dtype: int64

In [24]:
# 初始化分类决策树模型， 深度为5层
model = DecisionTreeClassifier(max_depth=6, random_state=1)

In [25]:
# 训练模型
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=1)

In [26]:
# 模型预测
y_pred = model.predict(X_test)

In [27]:
# 模型评估
accuracy_score(y_test, y_pred)

0.011315417256011316

In [33]:
# 将模型选择特征的标准改为entropy
model = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)

In [34]:
# 模型训练
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1)

In [35]:
# 预测
y_pred = model.predict(X_test)

In [36]:
# 评估
accuracy_score(y_test, y_pred)

0.0059554827663217446